In [1]:
%load_ext autoreload
%autoreload 2
from ee_extract import *
import geemap

ee.Initialize()
# 截取栾城的数据
# https://code.earthengine.google.com/0001f57d9a79be524076dbc10329538f

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
col = ee.ImageCollection("CAS/IGSNRR/PML/V2_v017")
img = col.first()
img

col_et = col.map(cal_ETsum)
col_et.limit(3)

## 截取数据


In [3]:
point = ee.Feature(ee.Geometry.Point([114 + 41/60, 37 + 53/60]), {'name': 'LuanCheng'})

In [4]:
filter = ee.Filter.date("2007-01-01", "2018-12-31")

def tidy_pml(id):
    return ee.ImageCollection(id).filter(filter).map(cal_ETsum)

col_pml_v014 = tidy_pml("projects/pml_evapotranspiration/PML/OUTPUT/PML_V2_8day_v014")
col_pml_v017 = tidy_pml("CAS/IGSNRR/PML/V2_v017")
col_pml_china = tidy_pml("users/shaoyanghe96/PMLV2_China_daily")

col_pml_china.limit(3)
# col_pml_v014 = col_pml_v014.map(cal_ETsum)

In [ ]:
%%time
col_export_points(point, col_pml_v014, "栾城_PMLV2_014")
col_export_points(point, col_pml_v017, "栾城_PMLV2_017")
col_export_points(point, col_pml_china, "栾城_PMLV2_China")


In [5]:
col_lai = ee.ImageCollection('MODIS/061/MOD15A2H') \
    .filter(filter) \
    .select(['Lai_500m'], ['LAI'])

vis_lai = {
    "min": 0, 
    "max": 20, 
    "palette": ["red", "blue", "green"]
}
# map = geemap.Map()
# map.addLayer(col_lai, vis_lai, "lai")
# map.add_colorbar(vis_lai, label="LAI", discrete=False);
# map

In [6]:
Map = geemap.Map()
Map.add_basemap('SATELLITE')
Map.centerObject(point, 14)
Map.addLayer(col_lai, vis_lai, "lai")
Map.add_colorbar(vis_lai, label="LAI", discrete=False);
Map.addLayer(point, {'color': 'black', 'pointsize': 12}, 'point')
Map.addLayer(point.buffer(1000), {'color': 'red', 'PointSize': 12}, '1km', True, 0.4)
Map

Map(center=[37.88333333333333, 114.68333333333334], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# ?ee.batch.Export.table.toDrive
# "CSV" (default), "GeoJSON", "KML", "KMZ", "SHP", or "TFRecord"
# ee.batch.Export.table.toDrive(res, "temp", folder="gee", fileFormat="GeoJSON").start()